# CS547 Group 17
Carlos Cuellar, Yuncong Geng, Kangcheng Lin, Peter Thompson
## HW5

In [120]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from copy import deepcopy

In [121]:
# Define some plotting styles that we will use.
fontSize=15
matplotlib.rcParams.update({"axes.formatter.limits": (-4,4),'font.family':'Avenir','font.size': fontSize, "axes.titlesize": fontSize, "xtick.labelsize": fontSize, "ytick.labelsize": fontSize,})
plotStyles={"markersize":12,"markeredgewidth":2.0,"linewidth":2.0}
stepStyles={"markersize":15,"markeredgewidth":3.0,"linewidth":4.0,"where":"post"}
scatterStyles={"linewidth":1.0}

## load data

In [122]:
df=pd.read_csv('HW5_data.csv', sep=',',index_col=0, header='infer')
df_xy = df[['X','Y']]
df_z = df[['Z']]

## split data

In [123]:
'''
def split_data(my_size): #random splitting
    training_idx = np.random.choice(range(0,my_size), int(0.6*my_size), replace=False);
    remaining_idx = np.setxor1d(range(0,my_size), training_idx);
    validation_idx = np.random.choice(remaining_idx, int(0.2*my_size), replace=False);
    testing_idx = np.setxor1d(remaining_idx, validation_idx);
    return (training_idx, validation_idx, testing_idx)
(training_idx, validation_idx, testing_idx) = split_data(len(df));
'''

my_size = len(df);
training_idx = np.array(range(0,int(0.6*my_size)));
validation_idx = np.array(range(int(0.6*my_size),int(0.8*my_size)));
testing_idx = np.array(range(int(0.8*my_size),int(1*my_size)));

## training and validation

In [131]:
MSE_best = 1e10
D_best = 0;
reg_best = LinearRegression();
for D in [2, 3, 4, 5, 6, 7]:
    poly = PolynomialFeatures(degree=D, include_bias=False); 
    
    #training
    XY = df_xy.iloc[training_idx,:];
    Z = df_z.iloc[training_idx,:].to_numpy();
    XY_poly = poly.fit_transform(XY);
    reg = LinearRegression(fit_intercept = True).fit(XY_poly, Z)
    z_pred = reg.predict(XY_poly);
    MSE_training = mean_squared_error(Z, z_pred);
    #validation
    
    XY_val = df_xy.iloc[validation_idx,:];
    Z_val = df_z.iloc[validation_idx,:].to_numpy();
    XY_poly_val = poly.fit_transform(XY_val);
    z_val_pred = reg.predict(XY_poly_val);
    MSE_val = mean_squared_error(Z_val, z_val_pred)
    if (MSE_val < MSE_best):
        MSE_best = MSE_val;
        D_best = D;
        reg_best = deepcopy(reg);
    print('D =', D, '\ttraining MSE = ', MSE_training,'\tvalidation MSE = ', MSE_val);
print('best D = ', D_best)

D = 2 	training MSE =  43430409.57778652 	validation MSE =  36271714.89597634
D = 3 	training MSE =  2066632.835603818 	validation MSE =  2180085.927675116
D = 4 	training MSE =  20544.828085888934 	validation MSE =  21960.615275848562
D = 5 	training MSE =  0.008270412119058674 	validation MSE =  0.010731902649748003
D = 6 	training MSE =  0.008132565722278669 	validation MSE =  0.011034240519792507
D = 7 	training MSE =  0.00795559028260813 	validation MSE =  0.011127754865817085
best D =  5


## testing

In [125]:
#testing
poly = PolynomialFeatures(degree=D_best,include_bias=False); 
XY_test = df_xy.iloc[testing_idx,:];
Z_test = df_z.iloc[testing_idx,:].to_numpy();
XY_poly_test = poly.fit_transform(XY_test);
z_test_pred = reg_best.predict(XY_poly_test);
MSE = mean_squared_error(Z_test, z_test_pred)
print('testing MSE =', MSE)

testing MSE = 0.010544438312310387


## coefficients for the best D

In [126]:
print(reg_best.coef_)

[[ 8.62706308e-01  9.25346591e-01  2.70539344e-01 -2.96455711e+00
   2.01698867e-02 -1.67192295e-02  4.93639101e-01 -6.24851806e-03
  -3.15132279e-03  3.01865573e-01  1.86021119e-04  9.04537822e-04
   4.52142630e-04  2.70267580e-04 -7.87628076e-05  2.49538014e-05
  -6.88331528e-05  8.92071001e-06 -2.92611205e-05  1.19999256e+00]]


In [127]:
print(reg_best.intercept_)

[5.12425338]
